In [1]:
import warnings
import datetime

import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns

from sklearn.model_selection import train_test_split
from IPython.display import display, Markdown , Math 

sns.set()
warnings.filterwarnings('ignore')

In [2]:
def printmd(string): display(Markdown(string))
def latex(out): printmd(f'{out}')  
def pr(string): printmd('***{}***'.format(string))

In [8]:
test = pd.read_csv("../../../resources/data/titanic/test.csv")
train = pd.read_csv("../../../resources/data/titanic/train.csv")

In [10]:

combine = pd.concat([train.drop('Survived',1),test])

In [12]:
train.shape

(891, 12)